In [1]:
import pandas as pd # import pandas for file manipulation, read and write, plus other functions
import yfinance as yf # import yfinance to pull stock data from the market
import os # import os to check if directory and file exist
from openpyxl import Workbook # import Workbook to create files
import time

pd.options.display.float_format = '{:.2f}'.format # display float numbers with 2 decimals  to help with visibility during coding

current_year = pd.Timestamp.today().year # define current year, so the workflow always creates a file per year

current_year # check current year

2025

In [ ]:
# Create an output file, if there's none

# Defining file path and name
diretorio = r'C:\Users\guilh\OneDrive\Investimentos'
arquivo = os.path.join(diretorio, f"{current_year} - Operações na Bolsa - IR.xlsx")

# Creating directory, if it doesn't exist
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Creating file, if it doesn't exist
if not os.path.isfile(arquivo):

    # Creating a new workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Operações na Bolsa"

    # Saving the file
    wb.save(arquivo)

    # Declaring file creation
    print(f"Arquivo criado e salvo em: {arquivo}")

else:
    # Informting file existance
    print(f"Arquivo já existe")


Arquivo já existe


In [3]:
# Read file with stocks operation

assets_df = pd.read_excel(fr"{diretorio}\Operações na Bolsa.xlsx") # read file with registered operations

# Sort dataframe by Operation Date in ascending order

assets_df = assets_df.sort_values(by='Data da Operação', ascending=True).reset_index(drop=True)

assets_df.head(5) # show sample

,Carimbo de data/hora,Data da Operação,Categoria,Ticker,Operação,Quantidade,Preço unitário,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Outros,Taxas - Operacional,Taxas - IRRF,Valor nominal total,Valor total com Taxas
0,2024-12-31 16:22:33.860,2024-01-10,FII,BTLG11,Compra,5,103.00,0.13,0.02,0,0.00,0,NaN,515.00,515.15
1,2024-12-31 16:22:33.860,2024-01-10,FII,LVBI11,Compra,4,118.65,0.13,0.02,0,0.00,0,0.00,474.60,474.75
2,2024-12-31 16:22:33.860,2024-01-10,FII,XPML11,Compra,5,115.80,0.13,0.03,0,0.00,0,0.00,579.00,579.16
3,2024-12-31 16:22:33.860,2024-01-25,FII,IRDM11,Compra,6,77.90,0.11,0.02,0,0.00,0,0.00,467.40,467.53
4,2024-12-31 16:22:33.860,2024-02-15,FII,VRTA11,Compra,5,87.50,0.11,0.02,0,0.00,0,0.00,437.50,437.63


# Retrieve value for each ticker

In [4]:
# Create df with tickers only

tickers = assets_df['Ticker'].drop_duplicates() # create a list with tickers for value searching

tickers.head(5) # show sample

0    BTLG11
1    LVBI11
2    XPML11
3    IRDM11
4    VRTA11
Name: Ticker, dtype: object

In [5]:
# Search for each ticker's value
price_list = [] # create empty price list

for ticker in tickers: # for loop to search for each ticker's value and fill out list
    data = yf.Ticker(f'{ticker}.SA').info # create variable to connect with yfinance library to retrieve ticker's information
    price = data.get('currentPrice') # store current price in variable 'price'
    price_list.append(price) # add ticker's price to price list

price_list[:5] # show sample

[102.65, 112.24, 108.06, 58.5, 82.6]

In [6]:
# Create df with asset price for later use

asset_price  = pd.DataFrame({'Ticker': tickers, 'Valor Unitário Atual': price_list}) # create df with price for each ticker, combining ticker list with price list

asset_price.head(5) # show sample

,Ticker,Valor Unitário Atual
0,BTLG11,102.65
1,LVBI11,112.24
2,XPML11,108.06
3,IRDM11,58.50
4,VRTA11,82.60


# Prepare Income Tax report

In [7]:
# Adjust quantity to negative when Sale, to consider actual current quantity for each asset

assets_df['Quantidade Transação'] = [(1 if row['Operação'] != 'Sale' else -1) * row['Quantidade'] # negative quantity if sale, to indicate reduction in portfolio
                                     for _, row in assets_df.iterrows()] # for every row in the df

assets_df[assets_df['Ticker'] == 'ITUB4']['Quantidade Transação'] # show sample


Series([], Name: Quantidade Transação, dtype: int64)

In [8]:
# Determine purchase group, so the Preço Médio Atual and quantity renew every time a ticker is completely sold and later purchased again

# First, create a cumulative count to transactioned quantity, that will later be dropped

assets_df['Quantidade Atual'] = assets_df.groupby('Ticker')['Quantidade Transação'].cumsum() # accumulate transactioned quantity per ticker

# Now, sum 1 to Purchase Group for every first purchase after a complete ticker sale, to calculate Preço Médio Atual and quantity correctly

assets_df['Grupo de Compra'] = 1 # stablish 1 to all purchase groups at first

for row in range(len(assets_df)): # iterate with each row
    if assets_df.loc[row, 'Quantidade Atual'] == 0: # will execute whenever acc transactioned quantity is 0
        ticker = assets_df.loc[row, 'Ticker'] # store ticker for that case
        for next in range (row + 1, len(assets_df)): # iterate with all upcoming rows
            if assets_df.loc[next, 'Ticker'] == ticker: # will execute whenever it finds a new operation with that ticker
                assets_df.loc[next, 'Grupo de Compra'] += 1 # sum 1 to purchase group, indicating it's a new group to calculte Preço Médio Atual and quantity


print(assets_df.loc[assets_df['Ticker'] == 'ITUB4', ['Quantidade Atual', 'Grupo de Compra']]) # print specific example

Empty DataFrame
Columns: [Quantidade Atual, Grupo de Compra]
Index: []


In [9]:
# Calculate buying price. Sales will be negative to represent revenue, and actual cost will naturally be less due to fees.

assets_df.loc[: , 'Preço Total'] = (assets_df['Preço unitário'] * assets_df['Quantidade Transação']) # Criar coluna Preço Total multiplicando Preço por Quantidade

# Calculate actual buying cost
assets_df.loc[: , 'Gasto Total'] = (assets_df['Preço Total'] # Calculate actual buying cost summing Preço Total with fees paid
+ assets_df['Taxas - Emolumentos']
+ assets_df['Taxas - IRRF']
+ assets_df['Taxas - Impostos']
+ assets_df['Taxas - Liquidação']
+ assets_df['Taxas - Operacional']
+ assets_df['Taxas - Outros']
)

assets_df.head(5) # show sample

,Carimbo de data/hora,Data da Operação,Categoria,Ticker,Operação,Quantidade,Preço unitário,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Outros,Taxas - Operacional,Taxas - IRRF,Valor nominal total,Valor total com Taxas,Quantidade Transação,Quantidade Atual,Grupo de Compra,Preço Total,Gasto Total
0,2024-12-31 16:22:33.860,2024-01-10,FII,BTLG11,Compra,5,103.00,0.13,0.02,0,0.00,0,NaN,515.00,515.15,5,5,1,515.00,NaN
1,2024-12-31 16:22:33.860,2024-01-10,FII,LVBI11,Compra,4,118.65,0.13,0.02,0,0.00,0,0.00,474.60,474.75,4,4,1,474.60,474.75
2,2024-12-31 16:22:33.860,2024-01-10,FII,XPML11,Compra,5,115.80,0.13,0.03,0,0.00,0,0.00,579.00,579.16,5,5,1,579.00,579.16
3,2024-12-31 16:22:33.860,2024-01-25,FII,IRDM11,Compra,6,77.90,0.11,0.02,0,0.00,0,0.00,467.40,467.53,6,6,1,467.40,467.53
4,2024-12-31 16:22:33.860,2024-02-15,FII,VRTA11,Compra,5,87.50,0.11,0.02,0,0.00,0,0.00,437.50,437.63,5,5,1,437.50,437.63


### Calculate average buy price after every transaction

This will be crucial for both calculating current buy price, as well as registering profit for each sale individually

In [10]:
# Calculate accumulated buy price without influence of Sales price, but accounting for the adjusted quantity after it with subsequent Preço Médio Atual rebalancing

# We'll create a for loop to iterate all rows in assets_df, that will take care of all necessary parameters and calculations

for i in range(len(assets_df)): # apply the following to each row in assets_df
    ticker = assets_df.loc[i, 'Ticker'] # create variable to store ticker for that row
    grupo_compra = assets_df.loc[i, 'Grupo de Compra'] # do the same for purchase group
    quant_trans = assets_df.loc[i, 'Quantidade Transação'] # do the same for transactioned quantity
    ttl_price = assets_df.loc[i, 'Preço Total'] # do the same for total price
    previous_rows = assets_df[(assets_df['Ticker'] == ticker) &
                              (assets_df['Grupo de Compra'] == grupo_compra) & 
                              (assets_df.index < i)] # create a sub-dataframe with all transactions in assets_df for the same ticker and purchase group, that happened before the current transaction

    # Since we'll need to pull data from previous rows, here we'll make sure not to break the code every first ticker and purchase group
    if previous_rows.empty: # apply if previous rows do not exist. that is: for every first transaction of each ticker and purchase group
        prev_cum_buy_price = 0 # stablish previous accumulated buy price as 0
        prev_cum_quant = 0 # stablish previous accumulated quantity as 0

    else: # if previous rows do exist, meaning all transactions after the first, now there is a baseline to compare
        prev_cum_buy_price = previous_rows['Preço Total Atual'].iloc[-1] # pull previous transactions' accumulated buy price
        prev_cum_quant = previous_rows['Quantidade Atual'].iloc[-1] # pull previous transactions' accumulated quantity
    
    # Now we'll calculate accumulated buy price, by pulling previous accumulated buy price and summing with current buy price. 
        # Bare in mind the process is fundamentally different between Sales and non-Sales transactions:
    if assets_df.loc[i, 'Operação'] != 'Sale': # apply if operation is not a Sale
        cum_buy_price = ttl_price + prev_cum_buy_price # accumulated buy price is total price plus previous row's accumulated buy price
    
    else: # apply if operation is Sale
        cum_buy_price = ((prev_cum_buy_price / prev_cum_quant) * quant_trans) + prev_cum_buy_price # accumulated buy price, but with a different approach:
        # pull previous row's average price and multiply by transactioned quantity to deduct from total accumulated buy price without changing avg price.
        # then sum resulted adjusted total price to previous accumulated buy price
    
    # Once we have the variable we want with the right values, all we need to do is bring it to the df
    assets_df.loc[i, 'Preço Total Atual'] = cum_buy_price # attribute values of variable with accumulated buy prices to column in assets_df


# Now that we have the accumulated buy price, we could just divide it by accumulated quantity to calculate average purchase price.
# However, that wouldn't work when the fully sell an asset, because this acc quantity goes to 0. So we'll go down a safer path:

# We'll iterate all rows from assets_df to make sure whenever we have a Sale, it will pull average price from previous row:
for i in range(len(assets_df)): # apply the following to each row in assets_df
    ticker = assets_df.loc[i, 'Ticker'] # create variable to store ticker for that row
    grupo_compra = assets_df.loc[i, 'Grupo de Compra'] # same for purchase group
    cum_buy_price = assets_df.loc[i, 'Preço Total Atual'] # same for accumulated buy price
    cum_quant = assets_df.loc[i, 'Quantidade Atual'] # same for accumulated quantity
    previous_rows = assets_df[(assets_df['Ticker'] == ticker) &
                              (assets_df['Grupo de Compra'] == grupo_compra) & 
                              (assets_df.index < i)] # create a sub-dataframe with all transactions in assets_df for the same ticker and purchase group, that happened before the current transaction

    # Since we'll need to pull data from previous rows, here we'll make sure not to break the code every first ticker and purchase group    
    if previous_rows.empty: # apply if previous rows do not exist. that is: for every first transaction of each ticker and purchase group
        prev_avg_price = 0 # stablish previous average price as 0
    
    else: # if previous rows do exist, meaning all transactions after the first, now there is a baseline to compare
        prev_avg_price = previous_rows['Preço Médio Atual'].iloc[-1] # pulls previous average price from previous row's average price column
    
    # As mentioned, the process is different for Sales. To avoid mistakes, we'll always pull average price from previous row's average price
    if assets_df.loc[i, 'Operação'] != 'Sale': # apply the following for non-Sales operations
        avg_price = cum_buy_price / cum_quant # average price is the division between accumulated buy price and accumulated quantity
    
    else: # apply the following for Sales operations
        avg_price = prev_avg_price # average price is the same as previous row's for the same ticker as purchase group

    # Once we have the variable we want with the right values, all we need to do is bring it to the df    
    assets_df.loc[i, 'Preço Médio Atual'] = avg_price    

assets_df.loc[assets_df['Ticker'] == 'ITUB4', ['Quantidade Atual','Preço Total Atual', 'Preço Médio Atual', 'Grupo de Compra']] # show specific example


,Quantidade Atual,Preço Total Atual,Preço Médio Atual,Grupo de Compra


In [11]:
# Export a detailed version of input file to the output file

file_path = fr"{diretorio}\{current_year} - Operações na Bolsa - IR.xlsx" # define path to file. it will be used for all dfs

assets_df_sheet_name = 'Operações na Bolsa' # define sheet name

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    assets_df.to_excel(writer, sheet_name=assets_df_sheet_name, index=False) # execute instance to replace the sheet

print('Arquivo exportado com sucesso') # confirm file export

Arquivo exportado com sucesso


## Latest Portfolio Overview

File with most current overview of the portfolio, to summarize all information that will ultimately be added to your income tax report

In [12]:
assets_df[assets_df['Ticker'] == 'ITUB4']

last_idx = assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())

last_idx.head(5)

C:\Users\guilh\AppData\Local\Temp\ipykernel_10412\3231979482.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_idx = assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())


Ticker  Grupo de Compra
BTLG11  1                   0
IRDM11  1                   9
LVBI11  1                  10
RECR11  1                  12
TGAR11  1                   8
dtype: int64

In [13]:
# Create dataframe that brings only the latest transaction for each Ticker and Purchase Group, with most recent Quantity, Average Price and Accumulated Buy Price

latest_assets_df = assets_df.loc[assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())] # create the dataframe grouping by ticker and purchase group, bringing only the row where the max index, since you already sorted by date in the beginning

# Clean dataframe to keep only useful columns and assets that are still available in portfolio. Completely sold assets do not go here.
final_assets_df = latest_assets_df.loc[latest_assets_df['Quantidade Atual'] > 0 , # bring only assets that were not completely sold
                                       ['Ticker', 
                                        'Categoria', 
                                        'Grupo de Compra', 
                                        'Quantidade Atual', 
                                        'Preço Médio Atual', 
                                        'Preço Total Atual']] # define useful columns to bring

final_assets_df = pd.merge(final_assets_df, asset_price, on= 'Ticker', how='inner') # bring current market price for each asset

final_assets_df.head()  # show sample

C:\Users\guilh\AppData\Local\Temp\ipykernel_10412\75189049.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  latest_assets_df = assets_df.loc[assets_df.groupby(['Ticker', 'Grupo de Compra']).apply(lambda x: x.index.max())] # create the dataframe grouping by ticker and purchase group, bringing only the row where the max index, since you already sorted by date in the beginning


,Ticker,Categoria,Grupo de Compra,Quantidade Atual,Preço Médio Atual,Preço Total Atual,Valor Unitário Atual
0,BTLG11,FII,1,5,103.00,515.00,102.65
1,IRDM11,FII,1,26,73.15,1901.89,58.50
2,LVBI11,FII,1,9,106.90,962.10,112.24
3,RECR11,FII,1,7,75.20,526.40,81.99
4,TGAR11,FII,1,5,88.75,443.74,93.00


In [14]:
# Calculate upside

# Calculate total current value of each asset
final_assets_df.loc[:, 'Valor Total Atual'] = final_assets_df['Valor Unitário Atual'] * final_assets_df['Quantidade Atual']

# Calculate financial upside by subtracting current total value and current total price
final_assets_df.loc[:, r'Upside (R$)'] = final_assets_df['Valor Total Atual'] - final_assets_df['Preço Total Atual']

# Calculate upside percentage by dividing current total value and current total price and subtracting 1
final_assets_df.loc[:, r'Upside %'] = final_assets_df['Valor Total Atual'] / final_assets_df['Preço Total Atual'] - 1

final_assets_df.head(5) # show sample

,Ticker,Categoria,Grupo de Compra,Quantidade Atual,Preço Médio Atual,Preço Total Atual,Valor Unitário Atual,Valor Total Atual,Upside (R$),Upside %
0,BTLG11,FII,1,5,103.00,515.00,102.65,513.25,-1.75,-0.00
1,IRDM11,FII,1,26,73.15,1901.89,58.50,1521.00,-380.89,-0.20
2,LVBI11,FII,1,9,106.90,962.10,112.24,1010.16,48.06,0.05
3,RECR11,FII,1,7,75.20,526.40,81.99,573.93,47.53,0.09
4,TGAR11,FII,1,5,88.75,443.74,93.00,465.00,21.26,0.05


In [15]:
# Drop assets already completely sold and export to excel

final_assets_df_sheet_name = 'Ativos em Patrimônio' # define sheet to register assets in portfolio

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    final_assets_df.to_excel(writer, sheet_name=final_assets_df_sheet_name, index=False) # execute instance to replace the sheet

print('O arquivo foi exportado com sucesso') # show sample

O arquivo foi exportado com sucesso


## Register sales that happened during the year

For each sale, bring average price on that occasion to calculate profit correctly and add it to the file

In [16]:
# Create df only with sales in current calendar year

sold_df = assets_df[(pd.to_datetime(assets_df['Data da Operação']).dt.year == current_year) & (assets_df['Operação'] == 'Sale')].drop(columns=[
    'Quantidade Transação', 
    'Quantidade Atual', 
    'Preço Total Atual']) # create df only with assets sold in current calendar year, without useless columns

sold_df.head(5) # show sample

,Carimbo de data/hora,Data da Operação,Categoria,Ticker,Operação,Quantidade,Preço unitário,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,Taxas - Outros,Taxas - Operacional,Taxas - IRRF,Valor nominal total,Valor total com Taxas,Grupo de Compra,Preço Total,Gasto Total,Preço Médio Atual


In [17]:
# Calculate gross and net profit

sold_df['Preço Total'] = abs(sold_df['Preço Total']) # convert Total Price to positive if negative. All sales have that column in negative.

sold_df['Gasto Total'] = abs(sold_df['Gasto Total']) # convert Total Spent to positive if negative.

# Calculate gross profit, by just subtracting total price and current total price, before fees
sold_df['Lucro Bruto'] = sold_df['Preço Total'] - (sold_df['Preço Médio Atual'] * sold_df['Quantidade'])

# Calculate net profit, by just subtracting total spent and current total price, after fees. This will be more useful for income purpuses.
sold_df['Lucro Líquido'] = sold_df['Gasto Total'] - (sold_df['Preço Médio Atual'] * sold_df['Quantidade'])

sold_df

,Carimbo de data/hora,Data da Operação,Categoria,Ticker,Operação,Quantidade,Preço unitário,Taxas - Liquidação,Taxas - Emolumentos,Taxas - Impostos,...,Taxas - Operacional,Taxas - IRRF,Valor nominal total,Valor total com Taxas,Grupo de Compra,Preço Total,Gasto Total,Preço Médio Atual,Lucro Bruto,Lucro Líquido


In [18]:
# Export df to excel file

sold_df_sheet_name = 'Vendas do Ano' # define sheet name

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer: # create instance to open file to replace sheet for new values
    sold_df.to_excel(writer, sheet_name=sold_df_sheet_name, index=False) # execute instance to replace the sheet

print('Arquivo exportado com sucesso') # confirm export

Arquivo exportado com sucesso
